# Image manipulation for GATE

## Contents:
* Reading images
* Merging images
* Dicom image to mhd
* Anonymising Dicom files

# Reading images

## Reading any binary image
PET mouse data from https://neuroimage.usc.edu/neuro/Digimouse_Download in Analyze (.hdr / .img) format.

In [ ]:
import numpy as np

file = 'data/mouse_PET.img'
data_type = np.float32
dimension = [128, 128, 129]

# read the binary file
image = np.fromfile(file, dtype=data_type)
image = image.reshape(dimension,order='F')

In [ ]:
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[63,:,:], cmap=plt.cm.jet)

In [ ]:
from ipywidgets import interact

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[myslice,:,:], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[0]-1))
    return;

im_viewer(image, plt.cm.jet)

## Reading Meta images (.mhd / .raw)
Easy image reading with the library SimpleITK

In [2]:
import SimpleITK as sitk

# 1) Read the image
itkimage = sitk.ReadImage('data/patient_CT.mhd')
image = sitk.GetArrayFromImage(itkimage)

In [ ]:
print ('itkimage.GetSize() :' , itkimage.GetSize())
print ('itkimage.GetSpacing() :' , itkimage.GetSpacing())

print('image.sum() =', image.sum())
print('image.min() =', image.min())
print('image.max() =', image.max())
print('image.shape :', image.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(image[140,:,:], cmap=plt.cm.bone)

## Exercise
* modify the image data so that the slices correspond to the last dimension
* save the modified image

In [ ]:
# complete here ...

In [ ]:
# image = image[::-1,:,:]
# output = sitk.GetImageFromArray(image)
# # sitk.WriteImage(output,workdir+'patient_CT.mhd')
# sitk.WriteImage(output,workdir+'patient_SPECT.mhd')

# Merging images

## Merge output DoseActor .mhd of splitted simulations

In [ ]:
import SimpleITK as sitk
from glob import glob

# directory of your simulation splitted outputs
output_dir = './'

# loop over the .mhd outputs
first_image = True
for file in glob(output_dir+'*Dose.mhd'):
#    if '_merged' not in file:
        print ('Reading file:',file)
        # 1) Read the image
        itkimage = sitk.ReadImage(file)
        # 2) Sum the image values
        if first_image:
            image_merged = sitk.GetArrayFromImage(itkimage)
            first_image = False
        else: 
            image_merged += sitk.GetArrayFromImage(itkimage)
        #print ('Sum of all voxels:',image_merged.max())

# 3) Write the sum of all images
itkimage_merged = sitk.GetImageFromArray(image_merged)
itkimage_merged.CopyInformation(itkimage)
sitk.WriteImage(itkimage_merged, output_dir+'output-Dose_merged.mhd')

print ('Merged file created:', output_dir+'output-Dose_merged.mhd')
print ('Dimension : ' , itkimage_merged.GetSize())
print ('Pixels Size :' , itkimage_merged.GetSpacing())

# Converting DICOM to MetaImage (.mhd/.raw)
* place your DICOM images (.dcm) in the folder convert/dicom 
* execute the cell below

It will convert your DICOM series into MetaImage files in the folder convert/mhd

In [ ]:
import SimpleITK as sitk

dirname = 'convert/dicom'

series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(dirname)

if not series_IDs:
    print('No series in directory ' + dirname + '\'')

for series in series_IDs:
    filename = series.split('.')[-1]
    sitk.WriteImage(sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(dirname, series)),
                    'mhd/'+filename+'.mhd')